In [418]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import lxml
import psycopg2
import argparse

In [419]:
from pgConnect import PgConnection
from requestLimiter import RequestLimiter
from config import Config
from typing import Dict

In [434]:
import pickle
# pickle.load(, 'r')
with open(r'./data/rl/basketball-reference.com.p', "rb") as input_file:
    e = pickle.load(input_file)
    
    

In [436]:
e.limit = 19

In [437]:
e._save()

Saving RequestLimiter status to disk...


In [229]:
def get_ith_table(soup, i, **kwargs):
    # Get and return table
    tables = soup.find_all('table', **kwargs)
    if len(tables) > 0:
        table = tables[i]
        return table
    else:
        print("No table found on this HTML page!")
        

In [230]:
config : Config = Config('config.ini')
# reader 
read_constants : Dict[str, str] = config.parse_section('reader')
BASE : str = read_constants['base']
NAME : str = BASE[BASE.find('.') + 1:]

# requestLimiter
rl_constants : Dict[str, str] = config.parse_section('requestLimiter')
load_loc = rl_constants['load_location']
LOAD_FILE : str = f'{load_loc}{NAME}.p'
INTERVAL : int = int(rl_constants['interval'])
LIMIT : int = int(rl_constants['limit'])

# ======
# 2. Parse args
# ======
YEAR : int = 2023

rl : RequestLimiter = RequestLimiter(BASE, 
                    interval = INTERVAL, 
                    limit = LIMIT, 
                    load = LOAD_FILE)
bases = {'summary_base' :BASE + f'/leagues/NBA_{YEAR}.html',
            'schedule_base' : BASE + '/leagues/NBA_%s_games-%s.html'}

Successfully loaded previous Rate Limiter info for https://www.basketball-reference.com
Initialized with 2 of 20 entries filled

Saving RequestLimiter status to disk...


In [364]:
# data = rl.get(requests.get, "https://www.basketball-reference.com")
# if not data:
#     print("Couldn't get information in learn_teams() function!")
# data = data.text
# soup = BeautifulSoup(data, 'html.parser')
print(type(get_ith_table(soup, 0)))

<class 'bs4.element.Tag'>


In [362]:
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [231]:
def learn_teams(link : str, rl : RequestLimiter) -> Dict[str, str]:
    tm_dict = {}
    data = rl.get(requests.get, link)
    if not data:
        print("Couldn't get information in learn_teams() function!")
        return 
    data = data.text
    soup = BeautifulSoup(data, 'html.parser')
    
    table = get_ith_table(soup, 4, class_ = 'stats_table')
    if table:
        rows = table.findChildren(['tr'])
        for row in rows:
            for a in row.find_all('a'):
                tm_dict[a.text] = BASE + a.get('href')
    else:
        print("Previously hit rate limit on website!")
    return tm_dict


In [232]:
def get_team_info(team : str, link : str, rl : RequestLimiter):
    data =rl.get(requests.get, link)
    if not data:
        print(f"Unable to retrieve team info for {team}!")
        return
    soup = BeautifulSoup(data.text, 'html.parser')
    arena = get_arena(soup)
    # roster = read_ith_table(soup, 0, id = 'roster')
    roster_table = get_ith_table(soup, 0, id = 'roster')
    return arena, roster_table

def get_arena(soup):
    # Find arena
    a = soup.find_all('div', id = 'meta')[0]
    p = a.find_all('p')[-1]
    arena = p.contents[2].strip()
    return arena

  
def read_ith_table(soup, i, **kwargs):
    table = get_ith_table(soup, i, **kwargs)
    if table:
        return pd.read_html(str(table), flavor='html5lib')[0]    


In [233]:
def make_team_tuple(tm, stadium):
    tm_tup = (YEAR, tm, stadium)
    return tm_tup

In [365]:
type(requests.get('http://www.espn.com'))

requests.models.Response

In [356]:
pgc = PgConnection(config)
conn = pgc.getConn()
cur = pgc.getCurs()

Connecting to the PostgreSQL database...


In [357]:
def process_debut_season(exp : str) -> int:
    num = int(exp.replace('R','0'))
    return YEAR - num

def process_height(ht : str) -> int:
    ht_split = ht.split('-')
    ft, inch = ht_split[0], ht_split[1]
    return int(ft) * 12 + int(inch)

def process_name(name : str) -> str:
    if name[-4:] == '(TW)':
        return name[:-4].strip()
    return name.strip()


In [358]:
def process_player_table(player_table):
    table = player_table
    thead = player_table.find('thead')
    data = []
    column_names = [th.text.strip() for th in thead.find_all('th')]
    had_link = set()
    for row in table.find_all(['tr']):
        row_data = []
        link_data = []
        for num, td in enumerate(row.find_all(['td','th'])):
            not_link = None
            if ''.join(td.text.strip()):
                not_link = ''.join(td.text.strip())
            row_data.append(not_link)

            if td.find('a'):
                link = td.a['href']
                link_data.append(link)
                had_link.add(num)
        set_cols = True
        data.append(row_data + link_data)

    had_link = list(had_link)
    had_link.sort()
    for val in had_link:
       column_names.append(column_names[val] + '_link')

    df = pd.DataFrame(data[1:], columns= column_names)
    return df


def process_rows_for_player(df):
    rows = []
    for num, row in df.iterrows():
        out = (process_name(row['Player']),
                row['Birth Date'],
                process_height(row['Ht']),
                int(row['Wt']),
                None,
                process_debut_season(row['Exp']),
                row[''].upper(),
                row['College'],
                row['Player_link'])
        rows.append(out)
    return rows

def process_rows_for_roster(df, tm):
    rows = []
    for num, row in df.iterrows():
        out = (YEAR,
                tm,
                process_name(row['Player']),
                row['Birth Date'],
                process_height(row['Ht']),
                row['Wt'],
                row['No.'],
                row['Pos'])
        rows.append(out)
    return rows

In [359]:
def players_to_db(tups):
    args = ','.join(cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8') for i in tups)
    qry = "INSERT INTO player VALUES " + (args) + " ON CONFLICT (player_name, dob, height, weight) DO NOTHING"
    # print(qry)
    try:
        cur.execute(qry)
        conn.commit()
        print("Commited player insertion!")
    except Exception as e:
        print("Couldn't execute and commit player insertion!")
        print(str(e))

def roster_to_db(tups):
    args = ','.join(cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8') for i in tups)
    qry = "INSERT INTO roster VALUES " + (args) + " ON CONFLICT (season, team, player_name, dob, height, weight) DO NOTHING"
    try:
        cur.execute(qry)
        conn.commit()
        print("Commited roster insertion!")
    except Exception as e:
        print("Couldn't execute and commit roster insertion!")
        print(str(e))

def team_to_db(team_tup):
    args = ','.join(cur.mogrify("(%s,%s,%s)", i).decode('utf-8') for i in team_tup)
    qry = "INSERT INTO team VALUES " + (args) + " ON CONFLICT (season, team) DO NOTHING"
    try:
        cur.execute(qry)
        conn.commit()
        print("Commited team insertion!")
    except Exception as e:
        print("Couldn't execute and commit team insertion!")
        print(str(e))


guy = None
def load_teams(year : int, bases : Dict[str, str], rl : RequestLimiter):
    team_links : Dict[str, str] = learn_teams(bases['summary_base'], rl)
    tl = dict((k, team_links[k]) for k in ['Boston Celtics','Miami Heat','Detroit Pistons'])
    print(tl)
    for tm, link in tl.items():
        stadium, player_table = get_team_info(tm, tl[tm], rl)

        team_tup = [(YEAR, tm, stadium)]
        team_to_db(team_tup)

        df = process_player_table(player_table)
        player_tups = process_rows_for_player(df)
        players_to_db(player_tups)
        
        roster_tups = process_rows_for_roster(df, tm)
        roster_to_db(roster_tups)

    return

In [360]:
tups = load_teams(year = YEAR, bases = bases, rl = rl)

Successfully processed append to queue...
Size of current queue... 1
Saving RequestLimiter status to disk...
{'Boston Celtics': 'https://www.basketball-reference.com/teams/BOS/2023.html', 'Miami Heat': 'https://www.basketball-reference.com/teams/MIA/2023.html', 'Detroit Pistons': 'https://www.basketball-reference.com/teams/DET/2023.html'}
Successfully processed append to queue...
Size of current queue... 2
Saving RequestLimiter status to disk...
Commited team insertion!
Commited player insertion!
Commited roster insertion!
Successfully processed append to queue...
Size of current queue... 3
Saving RequestLimiter status to disk...
Commited team insertion!
Commited player insertion!
Commited roster insertion!
Successfully processed append to queue...
Size of current queue... 4
Saving RequestLimiter status to disk...
Commited team insertion!
Commited player insertion!
Commited roster insertion!


In [208]:
print(type(args))

<class 'str'>


In [429]:
a = {'Boston Celtics': 'https://www.basketball-reference.com/teams/BOS/2023.html', 
'Sacramento Kings': 'https://www.basketball-reference.com/teams/SAC/2023.html', 
'Utah Jazz': 'https://www.basketball-reference.com/teams/UTA/2023.html', 
'New Orleans Pelicans': 'https://www.basketball-reference.com/teams/NOP/2023.html', 
'Los Angeles Lakers': 'https://www.basketball-reference.com/teams/LAL/2023.html', 
'Golden State Warriors': 'https://www.basketball-reference.com/teams/GSW/2023.html', 
'Denver Nuggets': 'https://www.basketball-reference.com/teams/DEN/2023.html', 
'Oklahoma City Thunder': 'https://www.basketball-reference.com/teams/OKC/2023.html', 
'Memphis Grizzlies': 'https://www.basketball-reference.com/teams/MEM/2023.html', 
'Indiana Pacers': 'https://www.basketball-reference.com/teams/IND/2023.html', 
'Atlanta Hawks': 'https://www.basketball-reference.com/teams/ATL/2023.html', 
'Chicago Bulls': 'https://www.basketball-reference.com/teams/CHI/2023.html', 
'Minnesota Timberwolves': 'https://www.basketball-reference.com/teams/MIN/2023.html', 
'Brooklyn Nets': 'https://www.basketball-reference.com/teams/BRK/2023.html', 
'New York Knicks': 'https://www.basketball-reference.com/teams/NYK/2023.html', 
'Phoenix Suns': 'https://www.basketball-reference.com/teams/PHO/2023.html', 
'Philadelphia 76ers': 'https://www.basketball-reference.com/teams/PHI/2023.html', 
'Dallas Mavericks': 'https://www.basketball-reference.com/teams/DAL/2023.html', 
'Portland Trail Blazers': 'https://www.basketball-reference.com/teams/POR/2023.html', 
'Milwaukee Bucks': 'https://www.basketball-reference.com/teams/MIL/2023.html', 
'Washington Wizards': 'https://www.basketball-reference.com/teams/WAS/2023.html', 
'San Antonio Spurs': 'https://www.basketball-reference.com/teams/SAS/2023.html', 
'Charlotte Hornets': 'https://www.basketball-reference.com/teams/CHO/2023.html', 
'Detroit Pistons': 'https://www.basketball-reference.com/teams/DET/2023.html', 
'Cleveland Cavaliers': 'https://www.basketball-reference.com/teams/CLE/2023.html', 
'Toronto Raptors': 'https://www.basketball-reference.com/teams/TOR/2023.html', 
'Orlando Magic': 'https://www.basketball-reference.com/teams/ORL/2023.html', 
'Houston Rockets': 'https://www.basketball-reference.com/teams/HOU/2023.html', 
'Los Angeles Clippers': 'https://www.basketball-reference.com/teams/LAC/2023.html', 
'Miami Heat': 'https://www.basketball-reference.com/teams/MIA/2023.html'}

In [430]:
rl = RequestLimiter(base_link = 'https://www.espn.com' ,
                    interval = 60, # in seconds
                    limit  = 20, 
                    load  = './data/rl/espn.com.p')

Successfully loaded previous Rate Limiter info for https://www.espn.com
Initialized with 0 of 20 entries filled

Saving RequestLimiter status to disk...


In [431]:
rl.get('https://www.espn.com', waitForPop = True)

TypeError: get() got an unexpected keyword argument 'waitForPop'

In [424]:
# import time
for i in range(20):
    time.sleep(1)
    rl.get(requests.get,'https://www.espn.com', True)

TypeError: get() takes 3 positional arguments but 4 were given

In [400]:
for i in range(20):
    time.sleep(1)
    rl.get(requests.get,'https://www.espn.com')

Successfully processed append to queue...
Size of current queue... 19
Saving RequestLimiter status to disk...
Successfully processed append to queue...
Size of current queue... 20
Saving RequestLimiter status to disk...
You're about to go over the limit, you'll have to wait!
Saving RequestLimiter status to disk...
You're about to go over the limit, you'll have to wait!
Saving RequestLimiter status to disk...


KeyboardInterrupt: 

In [385]:
if not ('a' and True):
    print('a')

In [373]:
set_a = set(a.keys())
for k in a.keys():
    print(f"Processing {k}...")
    set_a.remove(k)
print(set_a)


Processing Boston Celtics...
Processing Sacramento Kings...
Processing Utah Jazz...
Processing New Orleans Pelicans...
Processing Los Angeles Lakers...
Processing Golden State Warriors...
Processing Denver Nuggets...
Processing Oklahoma City Thunder...
Processing Memphis Grizzlies...
Processing Indiana Pacers...
Processing Atlanta Hawks...
Processing Chicago Bulls...
Processing Minnesota Timberwolves...
Processing Brooklyn Nets...
Processing New York Knicks...
Processing Phoenix Suns...
Processing Philadelphia 76ers...
Processing Dallas Mavericks...
Processing Portland Trail Blazers...
Processing Milwaukee Bucks...
Processing Washington Wizards...
Processing San Antonio Spurs...
Processing Charlotte Hornets...
Processing Detroit Pistons...
Processing Cleveland Cavaliers...
Processing Toronto Raptors...
Processing Orlando Magic...
Processing Houston Rockets...
Processing Los Angeles Clippers...
Processing Miami Heat...
set()


In [154]:
cur.execute("SELECT * FROM player")

In [100]:
ls

[('Derrick White',
  None,
  2018,
  76,
  190,
  'July 2, 1994',
  'US',
  'Colorado-Colorado Springs, Colorado',
  '/players/w/whitede01.html'),
 ('Sam Hauser',
  None,
  2022,
  80,
  215,
  'December 8, 1997',
  'US',
  'Marquette, Virginia',
  '/players/h/hausesa01.html')]

In [ ]:
def load_players(tm, df, rl):
    i = 0
    for index, row in df.iterrows():
        player_id = checkPlayerDatabase(row)
        if player_id is None:
            loadToPlayerDatabase(row, rl)
        loadToRoster()
        i += 1
        if i > 0:
            break
        # if player_id is None:
            # loadToPlayerDatabase()
        # loadToRoster(row, tm, player_id)
def loadToPlayerDatabase(row, rl):
    link = BASE + row['Player_link']
    player_tup = get_player_info(row)
